<a href="https://colab.research.google.com/github/Alexeijn/fiap-ml-prog-ia-python-capstone/blob/master/dataminning_whatsapptcesp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from pathlib import Path

In [0]:
%config IPCompleter.greedy=True

In [21]:
!pip install spacy
!python -m spacy download en
!python -m spacy download pt

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 21.2MB 3.9MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=b27e65e80b502afe9da086f4853a64c9fa7e9fcc627b0f02b491e053fed636ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-oiqbbuq7/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [4]:
!ls

sample_data


In [5]:
# O dataset no momento estará no Google Drive.

from google.colab import drive

ROOT = "/content/drive"

drive.mount(ROOT)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!ls "/content/drive/My Drive/whatsapptcesp"

tickets-20200401.csv


In [0]:
datasetbase = Path("/content/drive/My Drive/whatsapptcesp/")
dataset = pd.read_csv(datasetbase / "tickets-20200401.csv")

In [8]:
#Vamos ver extatamente quais são as colunas...
print(dataset.columns.tolist())

['Número do Chamado', 'Data', 'Assunto', 'De', 'E-mail de Origem', 'Prioridade', 'Departamento', 'Área de Atuação', 'Origem', 'Status Atual', 'Última Atualização', 'Data de Vencimento', 'Atrasado', 'Respondidos', 'Atribuído a', 'Agente Atribuído', 'Equipe Atribuída']


In [9]:
print()
print("Tamanho:")
print(dataset.shape)


Tamanho:
(1163, 17)


In [10]:
dataset.head(10)

,Número do Chamado,Data,Assunto,De,E-mail de Origem,Prioridade,Departamento,Área de Atuação,Origem,Status Atual,Última Atualização,Data de Vencimento,Atrasado,Respondidos,Atribuído a,Agente Atribuído,Equipe Atribuída
0,DTEC0000013053,2020-03-02 08:25:38,Autodesk,CLEVERSON FIUZA ALVES,cfalves@tce.sp.gov.br,Normal,DTEC 3T,Software / Problemas com aplicativos diversos,Phone,Fechado,2020-03-03 08:36:40,NaN,0,0,Junior Armando,Armando Junior,NaN
1,DTEC0000013059,2020-03-02 08:44:38,lentidao,DEVANI MATTIOLI SAMPAIO,dsampaio@tce.sp.gov.br,Normal,DTEC 3T,S.O. / Lentidão,Phone,Fechado,2020-03-02 09:44:15,NaN,0,1,Weigert Alexandre,Alexandre Weigert,NaN
2,DTEC0000013061,2020-03-02 08:49:53,Configurar macros no word,RAFAEL RODRIGUES,rafarodrigues@tce.sp.gov.br,Normal,DTEC 3T,Software / Instalação/configuração de Microsof...,Phone,Fechado,2020-03-02 16:16:01,NaN,0,0,Colpani Thomaz,Thomaz Colpani,NaN
3,DTEC0000013065,2020-03-02 08:55:35,Perfil PRODESP,Rosana Nishimoto das Chagas,gcccm@tce.sp.gov.br,Normal,DTEC 3T,Grande Porte / Configuração impressora de rela...,Phone,Fechado,2020-03-02 08:57:23,NaN,0,0,Colpani Thomaz,Thomaz Colpani,NaN
4,DTEC0000013066,2020-03-02 08:56:47,Aviso de cota excedida',NIELSEN ELISA DE MORAIS DOS SANTOS,nmorais@tce.sp.gov.br,Normal,DTEC 3T,E-mail / Problemas com Outlook/Webmail,Phone,Fechado,2020-03-03 11:09:41,NaN,0,0,Matheus Grillo Rogério,Rogério Matheus Grillo,NaN
5,USR0000002714,2020-03-02 09:22:39,Problema,DENISE NEGRINI,dnegrini@tce.sp.gov.br,Normal,DTEC 3T,"08.2 De softwares e aplicativos comuns (Word, ...",Phone,Fechado,2020-03-04 14:57:23,NaN,0,0,Colpani Thomaz,Thomaz Colpani,NaN
6,DTEC0000013075,2020-03-02 09:46:59,Instalar aplicativo,MANUELA PRADO LEITÃO,mleitao@tce.sp.gov.br,Normal,DTEC 3T,Software / Instalação/configuração de aplicati...,Phone,Fechado,2020-03-03 10:36:20,NaN,0,0,Colpani Thomaz,Thomaz Colpani,NaN
7,DTEC0000013076,2020-03-02 09:48:19,Instalar app,MARCIO EDUARDO ARANTES PEREIRA,meapereira@tce.sp.gov.br,Normal,DTEC 3T,Software / Instalação/configuração de aplicati...,Phone,Fechado,2020-03-03 16:08:11,NaN,0,0,Colpani Thomaz,Thomaz Colpani,NaN
8,DTEC0000013080,2020-03-02 10:13:09,Instalação de impressoras,Bruno Luiz Rodrigues Da Silva,blsilva@tce.sp.gov.br,Normal,DTEC 3T,Grande Porte / Configuração impressora de rela...,Phone,Fechado,2020-03-02 15:48:19,NaN,0,0,Matheus Grillo Rogério,Rogério Matheus Grillo,NaN
9,DTEC0000013081,2020-03-02 10:29:19,Problemas para entrar no SEI,Loadyr da Silva Junior,lsjunior@tce.sp.gov.br,Normal,DTEC 3T,Software / Problemas com sistemas internos,Phone,Fechado,2020-03-03 14:05:16,NaN,0,0,Gabriel Pereira Biolcatti Gustavo,Gustavo Gabriel Pereira Biolcatti,NaN


In [11]:
#Análise de valores inválidos
print(dataset.isna().any())

Número do Chamado     False
Data                  False
Assunto               False
De                    False
E-mail de Origem      False
Prioridade            False
Departamento          False
Área de Atuação       False
Origem                False
Status Atual          False
Última Atualização    False
Data de Vencimento     True
Atrasado              False
Respondidos           False
Atribuído a            True
Agente Atribuído       True
Equipe Atribuída       True
dtype: bool


In [0]:
# As colunas que importam não tem valores nulos, podemos fazer uma análise então.
#Muitas vezes a Coluna "Assunto" é muito mais útil que a "Área de Atuação"
#Vou criar um dataset mesclando essas duas colunas
#dataset_novo["pedido"] = dataset["Assunto"] + dataset["Área de Atuação"]
#dataset_novo = pd.concat([dataset["Assunto"]], dataset["Área de Atuação"])
#dataset_novo.head()
#dataset["Assunto"].

In [78]:
palavras_assunto = dataset["Assunto"].str.cat(sep=" ").lower()
len(palavras_assunto)

29471

In [0]:
import nltk
import spacy

nlp = spacy.load('pt')
nlp.Defaults.stop_words |= {"instalação", "acesso", "configuração", "instalar", "problema", "problemas", "orientação", "o", "a", "e"}

In [0]:
doc = nlp(palavras_assunto)

In [0]:
sem_stop_words = [(token.orth_, token.lemma_) for token in doc if not (token.is_stop or token.is_punct)]

In [151]:
len(sem_stop_words)

2575

In [152]:
sem_stop_words[0:10]
tmp_dataframe = pd.DataFrame(data=sem_stop_words,columns=("Orth", "Lemma"))
tmp_dataframe.head()
tmp_dataframe[tmp_dataframe["Lemma"]=="o"]

,Orth,Lemma
32,a,o
34,a,o
35,o,o
67,a,o
68,o,o
...,...,...
2555,a,o
2556,o,o
2558,a,o
2563,a,o


In [0]:
from collections import Counter

In [0]:
word_freq = Counter(sem_stop_words)
common_words = word_freq.most_common(10)

In [155]:
print(common_words)

[(('ergon', 'ergon'), 150), (('o', 'o'), 127), (('vpn', 'vpn'), 125), (('a', 'o'), 123), (('pidgin', 'pidgin'), 83), (('remoto', 'remoto'), 67), (('cisco', 'ciscar'), 55), (('impressora', 'impressor'), 48), (('pdf', 'pdf'), 37), (('nota', 'noto'), 35)]
